In [3]:
import os
import json
import numpy as np
import pandas as pd
from collections import deque

def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - \
              np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    return 360.0 - angle if angle > 180 else angle

def smooth_angle(angle_queue, new_angle, window_size=5):
    angle_queue.append(new_angle)
    if len(angle_queue) > window_size:
        angle_queue.popleft()
    return np.mean(angle_queue)

def extract_landmark(frame, name):
    for lm in frame.get("landmarks", []):
        if lm["name"] == name and lm.get("visibility", 1.0) > 0.5:
            return [lm["x"], lm["y"]]
    return None

def count_deadlift_reps_from_json(json_path,
                                  angle_threshold_down=100,
                                  angle_threshold_up=160,
                                  min_frame_gap=15):
    with open(json_path, 'r') as f:
        frames = json.load(f)

    reps = 0
    frame_idx = 0
    last_rep_frame = -min_frame_gap
    stage = None
    angle_queue = deque(maxlen=5)
    frame_reps = []

    for frame in frames:
        r_shoulder = extract_landmark(frame, "right_shoulder")
        r_hip = extract_landmark(frame, "right_hip")
        r_knee = extract_landmark(frame, "right_knee")

        if None in [r_shoulder, r_hip, r_knee]:
            frame_idx += 1
            continue

        raw_angle = calculate_angle(r_shoulder, r_hip, r_knee)
        smoothed_angle = smooth_angle(angle_queue, raw_angle)

        if smoothed_angle < angle_threshold_down and stage != "down":
            stage = "down"
        elif smoothed_angle > angle_threshold_up and stage == "down":
            if frame_idx - last_rep_frame > min_frame_gap:
                reps += 1
                frame_reps.append(frame_idx)
                last_rep_frame = frame_idx
                stage = "up"

        frame_idx += 1

    return reps, frame_reps

def process_deadlift_json_folder(folder_path, output_excel='rep_counts_deadlift_json.xlsx'):
    results = []
    json_files = [f for f in os.listdir(folder_path) if f.endswith('.json')]

    for json_file in json_files:
        json_path = os.path.join(folder_path, json_file)
        try:
            rep_count, rep_frames = count_deadlift_reps_from_json(json_path)
            result = {
                "Video_Name": json_file,
                "Total_Reps": rep_count
            }
            for i, frame in enumerate(rep_frames):
                result[f"Rep_{i+1}_Frame"] = frame
            results.append(result)
            print(f"{json_file}: {rep_count} reps")
        except Exception as e:
            print(f"Error processing {json_file}: {e}")
            results.append({"Video_Name": json_file, "Total_Reps": 0})

    df = pd.DataFrame(results)
    output_path = os.path.join(folder_path, output_excel)
    df.to_excel(output_path, index=False)
    print(f"\n✅ Results saved to {output_path}")
    return df

# Run
if __name__ == "__main__":
    json_dir = r"C:\Users\bdsid\OneDrive\Desktop\Dune Tech\Rep Counting\Data\Test_Bhaskar_Sir\deadlift\Json"
    process_deadlift_json_folder(json_dir)


deadlift_segment_7_start_25m17s329ms_pose.json: 1 reps

✅ Results saved to C:\Users\bdsid\OneDrive\Desktop\Dune Tech\Rep Counting\Data\Test_Bhaskar_Sir\deadlift\Json\rep_counts_deadlift_json.xlsx
